In [1]:
import os
import sys
import json


os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.WFGenerator import *
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
import joblib
from joblib import Parallel, delayed
import superrad
from superrad import ultralight_boson

In [3]:
#number of cpus to use for parallelization
NCPUs = 4

#data directory relative to local parent GWGen
DataDirectory = "/home/shaunf/tmp/"
#DataDirectory = "/remote/pi213f/fell/DataStore/ProcaAroundKerrGW/GWGenOutput/"

#generate plots
PlotData = True

#boson spin
spin=1

#parameters
BHSpin=0.9 #SMBH Spin
p0=10. #Initial Semilatus Rectum
e0=0.2 #Initial Eccentricity
x0=1. #Initial Inclincation
qS=np.pi/4 #Sky Location Polar Angle
phiS=0. #Sky Location Azimuthal Angle
qK=1e-6 #Initial BH Spin Polar Angle. We want this to be as close to zero as allowed by FEW package. This must zero so the secondary BH orbits on equator of SMBH
phiK=0. #Initial BH Spin Azimuthal Angle
dist=1. #Distance to source (Mpc)
mich=False #assume LISA long baseline response approximation

T=5 #LISA data run
dt=15 #time resolution in seconds

use_gpu=False #if CUDA or cupy is installed, this flag sets GPU parallelization


# keyword arguments for inspiral generator (RunKerrGenericPn5Inspiral)
inspiral_kwargs = {
    "npoints": 100,  # we want a densely sampled trajectory
    "max_init_len": int(1e3),
}

# keyword arguments for summation generator (AAKSummation)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}

unmoddedwvcl = EMRIWaveform(inspiral_kwargs=inspiral_kwargs, sum_kwargs=sum_kwargs, use_gpu=False)
moddedwvcl = EMRIWithProcaWaveform(inspiral_kwargs=inspiral_kwargs, sum_kwargs=sum_kwargs, use_gpu=False)
ulb = superrad.ultralight_boson.UltralightBoson(spin=1, model="relativistic")

In [4]:
unmoddedwvcl = EMRIWaveform(inspiral_kwargs=inspiral_kwargs.copy(), sum_kwargs=sum_kwargs.copy(), use_gpu=False)
moddedwvcl = EMRIWithProcaWaveform(inspiral_kwargs=inspiral_kwargs.copy(), sum_kwargs=sum_kwargs.copy(), use_gpu=False)    

In [10]:
def process(BHMASS, PROCAMASS, plot=False,alphauppercutoff=0.335, alphalowercutoff=0.06,SecondaryMass=10, DataDir = DataDirectory):

    alphaval = alphavalue(BHMASS, PROCAMASS)
    #alpha values larger than 0.02 produce energy fluxes larger than the undressed flux
    if alphaval>alphauppercutoff and spin==1:
        return None
    if alphaval<alphalowercutoff and spin==1:
        return None

    print("Alpha Value: {2}\nSMBH Mass: {0}\nProca Mass: {1}".format(BHMASS, PROCAMASS,alphaval))

    #Important: only pass copied version of kwargs as class can overwrite global variables. Should fix this....
    unmoddedwvcl = EMRIWaveform(inspiral_kwargs=inspiral_kwargs.copy(), sum_kwargs=sum_kwargs.copy(), use_gpu=False)
    moddedwvcl = EMRIWithProcaWaveform(inspiral_kwargs=inspiral_kwargs.copy(), sum_kwargs=sum_kwargs.copy(), use_gpu=False)

    moddedwv = moddedwvcl(BHMASS, SecondaryMass, PROCAMASS, BHSpin,p0,e0,x0,T=T, qS=qS, phiS=phiS, qK=qK, phiK=phiK, dist=dist,mich=mich,dt=dt, BosonSpin=spin, UltralightBoson = ulb)
    moddedtraj = moddedwvcl.Trajectory





    return None

In [13]:
SMBHMasses = [1e5]*1000000
ProcaMasses = [8e-17]*10000
Parallel(n_jobs=NCPUs, backend="multiprocessing")(delayed(process)(bhmass, pmass,plot=PlotData, SecondaryMass=SecondaryMass) for bhmass in SMBHMasses for pmass in ProcaMasses)


KeyboardInterrupt: 